In [3]:
import numpy as np
import json
import argparse
import glob
import sys
import shutil

from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import selections
from enterprise.signals import white_signals
from enterprise.signals.gp_signals import MarginalizingTimingModel
from enterprise_extensions import blocks
from troj_signal_a_term import troj_res_block
from enterprise.signals import signal_base

from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

import pint.models.model_builder as mb

In [4]:
# parser = argparse.ArgumentParser()
# parser.add_argument("-pname", "--pulsar_name")
# parser.add_argument("-niter", "--num_of_iteration", default=int(1e6))

# args = parser.parse_args()
pname = "J0023+0923"
# pname = args.pulsar_name
niter = 5
# niter = int(args.num_of_iteration)

print(f"Working on PSR {pname}.")

par = glob.glob(f"./data/par/{pname}_PINT_*.nb.par")
tim = glob.glob(f"./data/tim/{pname}_PINT_*.nb.tim")
if len(par) != 1 and len(tim) != 1:
    sys.exit("Exiting! More than one par/tim file present for this pulsar!")
psr = Pulsar(par[0], tim[0], timing_package="pint")

tmin = psr.toas.min()
tmax = psr.toas.max()
Tspan = tmax - tmin
print(f"Tspan for {pname} =  {Tspan/(365.25*86400)} years.")

Working on PSR J0023+0923.


2025-07-03 17:21:54.289 | DEBUG    | pint.toa:get_TOAs:195 - Using EPHEM = DE440 from the given model
2025-07-03 17:21:54.291 | DEBUG    | pint.toa:get_TOAs:211 - Using CLOCK = BIPM2019 from the given model
2025-07-03 17:22:05.041 | DEBUG    | pint.toa:apply_clock_corrections:2224 - Applying clock corrections (include_bipm = True)
2025-07-03 17:22:06.821 | INFO     | pint.observatory:gps_correction:235 - Applying GPS to UTC clock correction (~few nanoseconds)
2025-07-03 17:22:06.822 | DEBUG    | pint.observatory:_load_gps_clock:109 - Loading global GPS clock file
2025-07-03 17:22:06.855 | INFO     | pint.observatory.global_clock_corrections:get_file:129 - File index.txt to be downloaded due to download policy if_expired: https://raw.githubusercontent.com/ipta/pulsar-clock-corrections/main/index.txt
2025-07-03 17:22:07.042 | DEBUG    | pint.observatory.clock_file:__init__:812 - Global clock file gps2utc.clk saving kwargs={'bogus_last_correction': False, 'valid_beyond_ends': False}
2025-

Tspan for J0023+0923 =  9.01592586048203 years.


In [2]:
psr.setpars

['PSR',
 'TRACK',
 'EPHEM',
 'CLOCK',
 'UNITS',
 'START',
 'FINISH',
 'RM',
 'INFO',
 'TIMEEPH',
 'T2CMETHOD',
 'BINARY',
 'DILATEFREQ',
 'DMDATA',
 'NTOA',
 'CHI2',
 'TZRMJD',
 'TZRSITE',
 'TZRFRQ',
 'PEPOCH',
 'POSEPOCH',
 'ECL',
 'CORRECT_TROPOSPHERE',
 'PLANET_SHAPIRO',
 'NE_SW',
 'SWP',
 'SWM',
 'DM',
 'DM1',
 'DMEPOCH',
 'DMX',
 'DMXR1_0001',
 'DMXR2_0001',
 'DMXR1_0002',
 'DMXR2_0002',
 'DMXR1_0003',
 'DMXR2_0003',
 'DMXR1_0004',
 'DMXR2_0004',
 'DMXR1_0005',
 'DMXR2_0005',
 'DMXR1_0006',
 'DMXR2_0006',
 'DMXR1_0007',
 'DMXR2_0007',
 'DMXR1_0008',
 'DMXR2_0008',
 'DMXR1_0009',
 'DMXR2_0009',
 'DMXR1_0010',
 'DMXR2_0010',
 'DMXR1_0011',
 'DMXR2_0011',
 'DMXR1_0012',
 'DMXR2_0012',
 'DMXR1_0013',
 'DMXR2_0013',
 'DMXR1_0014',
 'DMXR2_0014',
 'DMXR1_0015',
 'DMXR2_0015',
 'DMXR1_0016',
 'DMXR2_0016',
 'DMXR1_0017',
 'DMXR2_0017',
 'DMXR1_0018',
 'DMXR2_0018',
 'DMXR1_0019',
 'DMXR2_0019',
 'DMXR1_0020',
 'DMXR2_0020',
 'DMXR1_0021',
 'DMXR2_0021',
 'DMXR1_0022',
 'DMXR2_0022',
 'DM

In [33]:
import pint.models.model_builder as mb

In [5]:
pname = "J0023+0923"
par = glob.glob(f"./data/par/{pname}_PINT_*.nb.par")
tim = glob.glob(f"./data/tim/{pname}_PINT_*.nb.tim")
model = mb.get_model(par[0])

In [68]:
model.FB0.value is None

True

In [17]:
tmin = psr.toas.min()
tmax = psr.toas.max()
t_avg = (tmin + tmax)/2
print(f"{t_avg=}")
print(f"{t_avg/86400=}")

t_avg=4959781740.023523
t_avg/86400=57404.88125027226


In [19]:
print(model.TASC.value * 86400)
model.TASC

4959788243.578815873


MJDParameter(   TASC                57404.9565229029614919 (d) +/- 4.4158779086007055e-08 d frozen=False)

In [8]:
8.338e-5 * 86400 * np.pi*2

45.264268014851574

In [9]:
parameter.Constant()

enterprise.signals.parameter.Constant.<locals>.Constant

In [10]:
x= 3

In [11]:
x= parameter.Constant(23.3)("x")

In [12]:
x.value

23.3

In [13]:
x*2

TypeError: unsupported operand type(s) for *: 'Constant' and 'int'

In [ ]:
float(x)

In [20]:
import numpy as np
import json
import argparse
import glob
import sys
import shutil

from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals.parameter import Uniform
from enterprise.signals import selections
from enterprise.signals import white_signals
from enterprise.signals.gp_signals import MarginalizingTimingModel
from enterprise_extensions import blocks
from troj_signal import troj_res_block
from enterprise.signals import signal_base

from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
import pint.models.model_builder as mb

# parser = argparse.ArgumentParser()
# parser.add_argument("-pname", "--pulsar_name", type=str)
# parser.add_argument("-mc_min", "--m_c_min_mass", type=float,)
# parser.add_argument("-niter", "--num_of_iteration", default=int(1e6), type=int)

# args = parser.parse_args()
# pname = args.pulsar_name
# niter = args.num_of_iteration
# mc_min = args.m_c_min_mass

# args = parser.parse_args()
pname = "J0023+0923"
# pname = args.pulsar_name
niter = 5
mc_min = 0.016388
# niter = int(args.num_of_iteration)

print(f"Working on PSR {pname}.")

par = glob.glob(f"./data/par/{pname}_PINT_*.nb.par")
tim = glob.glob(f"./data/tim/{pname}_PINT_*.nb.tim")
if len(par) != 1 and len(tim) != 1:
    sys.exit("Exiting! More than one par/tim file present for this pulsar!")
psr = Pulsar(par[0], tim[0], timing_package="pint")
model = mb.get_model(par[0])
f_b = model.FB0.value # no factor of 86400 because ENTERPRISE uses units of seconds for TOAsa
n_b_Const = parameter.Constant(f_b * np.pi*2)("n_b")
n_b = f_b * np.pi*2
# n_b = f_b * np.pi*2

kappa = (1.4 - mc_min) / (1.4 + mc_min)
nu_min = 1/10 * n_b/2 * np.sqrt(2 - np.sqrt(27*kappa**2 - 23))

tmin = psr.toas.min()
tmax = psr.toas.max()
Tspan = tmax - tmin
print(f"Tspan for {pname} =  {Tspan/(365.25*86400)} years.")

# define selection by observing backend
selection = selections.Selection(selections.by_backend)

# white noise parameters
efac = parameter.Constant()
equad = parameter.Constant()
ecorr = parameter.Constant()



efeq = white_signals.MeasurementNoise(
    efac=efac, log10_t2equad=equad, selection=selection
)
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection)

tm = MarginalizingTimingModel(use_svd=True)
rn = blocks.red_noise_block(psd="powerlaw", components=30)
troj = troj_res_block(nu=Uniform(nu_min, n_b * np.sqrt(2)/2), n_b = n_b_Const)

signal_model = tm + efeq + ec + rn + troj
pta = signal_base.PTA(signal_model(psr))

noisedict = json.load(open("./data/15year_noise_dict.json", "r"))

# set white noise parameters with dictionary
pta.set_default_params(noisedict)
print(pta.summary())
print(f"param names = {pta.param_names}")

x0 = np.hstack([p.sample() for p in pta.params])
print(f"x0 = {x0}")
print(f"lnprior at x0 = {pta.get_lnprior(x0)}")
print(f"lnlikelihood at x0 = {pta.get_lnlikelihood(x0)}")

ndim = len(x0)

# set up the sampler:
# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2)

# chaindir = f"/lorule/scratch/jdt00012/chains_7Nov_24/{psr.name}/"

# sampler = ptmcmc(
#     ndim,
#     pta.get_lnlikelihood,
#     pta.get_lnprior,
#     cov,
#     outDir=chaindir,
#     resume=False,
# )

# sampler.sample(x0, niter)
# shutil.move(f"{pname}.out", f"{chaindir}/{pname}.out")

Working on PSR J0023+0923.


SystemExit: Exiting! More than one par/tim file present for this pulsar!

In [ ]:
print(pta.summary())
print(f"param names = {pta.param_names}")

In [4]:
pname = "J0023+0923"
# pname = args.pulsar_name
niter = 5
mc_min = 0.016388
# niter = int(args.num_of_iteration)

print(f"Working on PSR {pname}.")

par = glob.glob(f"./data/par/{pname}_PINT_*.nb.par")
tim = glob.glob(f"./data/tim/{pname}_PINT_*.nb.tim")
# if len(par) != 1 and len(tim) != 1:
#     sys.exit("Exiting! More than one par/tim file present for this pulsar!")
# psr = Pulsar(par[0], tim[0], timing_package="pint")
model = mb.get_model(par[0])
f_b = model.FB0.value # no factor of 86400 because ENTERPRISE uses units of seconds for TOAsa
n_b_Const = parameter.Constant(f_b * np.pi*2)("n_b")
n_b = f_b * np.pi*2

Working on PSR J0023+0923.


In [5]:
n_b * np.sqrt(2)/2

0.000370479639780541989

In [9]:
kappa = (1.4 - mc_min) / (1.4 + mc_min)
nu_expected = n_b/2 * np.sqrt(2 - np.sqrt(27*kappa**2 - 23))
nu_min = 1/10 * nu_expected
print(f"{nu_min=}")
print(f"{nu_expected=}")

nu_min=1.5212479756635923759e-05
nu_expected=0.00015212479756635922914


In [2]:
import numpy as np
import json
import glob
import sys
import shutil

from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals.parameter import Uniform
from enterprise.signals import selections
from enterprise.signals import white_signals
from enterprise.signals.gp_signals import MarginalizingTimingModel
from enterprise_extensions import blocks
from troj_signal import troj_res_block
from enterprise.signals import signal_base

from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
import pint.models.model_builder as mb

pname = "J0023+0923_synthetic_injected4"
mc_min = 0.016388
synthetic = True

print(f"Working on PSR {pname}.")

if synthetic:
    par = glob.glob(f"./red_noise_and_injected/{pname}.par")
    tim = glob.glob(f"./red_noise_and_injected/{pname}.tim")
    psr_real_name = "J0023+0923"
else:
    par = glob.glob(f"./data/par/{pname}_PINT_*.nb.par")
    tim = glob.glob(f"./data/tim/{pname}_PINT_*.nb.tim")
    psr_real_name = psr
    

if len(par) != 1 and len(tim) != 1:
    sys.exit("Exiting! More than one par/tim file present for this pulsar!")
psr = Pulsar(par[0], tim[0], timing_package="pint")
model = mb.get_model(par[0])
f_b = model.FB0.value # no factor of 86400 because ENTERPRISE uses units of seconds for TOAs
if f_b is None:
    P_b_days = model.PB.value
    P_b_seconds = P_b_days * 86400
    f_b = 1 / P_b_seconds

n_b_Const = parameter.Constant(f_b * np.pi*2)("n_b")
n_b = f_b * np.pi*2
# n_b = f_b * np.pi*2

kappa = (1.4 - mc_min) / (1.4 + mc_min)
nu_min = 1/10 * n_b/2 * np.sqrt(2 - np.sqrt(27*kappa**2 - 23))
nu_max = n_b * np.sqrt(2)/2*1.1

tmin = psr.toas.min()
tmax = psr.toas.max()
Tspan = tmax - tmin
print(f"Tspan for {pname} =  {Tspan/(365.25*86400)} years.")

# define selection by observing backend
selection = selections.Selection(selections.by_backend)

# white noise parameters
efac = parameter.Constant()
equad = parameter.Constant()
ecorr = parameter.Constant()



efeq = white_signals.MeasurementNoise(
    efac=efac, log10_t2equad=equad, selection=selection
)
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection)

tm = MarginalizingTimingModel(use_svd=True)
rn = blocks.red_noise_block(psd="powerlaw", components=30)
troj = troj_res_block(nu=Uniform(nu_min, nu_max), n_b = n_b_Const)

signal_model = tm + efeq + ec + rn + troj
pta = signal_base.PTA(signal_model(psr))

noisedict = json.load(open("./data/15year_noise_dict.json", "r"))

# set white noise parameters with dictionary
pta.set_default_params(noisedict)
print(pta.summary())
print(f"param names = {pta.param_names}")
print(f"nu_min = {nu_min}, nu_max = {nu_max}")

x0 = np.hstack([p.sample() for p in pta.params])
print(f"x0 = {x0}")
print(f"lnprior at x0 = {pta.get_lnprior(x0)}")
print(f"lnlikelihood at x0 = {pta.get_lnlikelihood(x0)}")

ndim = len(x0)

# set up the sampler:
# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2)



Working on PSR J0023+0923_synthetic_injected4.


2025-02-14 16:12:25.765 | DEBUG    | pint.toa:get_TOAs:195 - Using EPHEM = DE440 from the given model
2025-02-14 16:12:25.766 | DEBUG    | pint.toa:get_TOAs:211 - Using CLOCK = BIPM2019 from the given model
2025-02-14 16:12:36.281 | DEBUG    | pint.toa:apply_clock_corrections:2224 - Applying clock corrections (include_bipm = True)
2025-02-14 16:12:38.134 | INFO     | pint.observatory:gps_correction:235 - Applying GPS to UTC clock correction (~few nanoseconds)
2025-02-14 16:12:38.135 | DEBUG    | pint.observatory:_load_gps_clock:109 - Loading global GPS clock file
2025-02-14 16:12:38.146 | DEBUG    | pint.observatory.clock_file:__init__:812 - Global clock file gps2utc.clk saving kwargs={'bogus_last_correction': False, 'valid_beyond_ends': False}
2025-02-14 16:12:38.152 | DEBUG    | pint.observatory.clock_file:read_tempo2_clock_file:463 - Loading TEMPO2-format observatory clock correction file gps2utc.clk (/minish/jdt00012/.astropy/cache/download/url/d3c81b5766f4bfb84e65504c8a453085/cont

Tspan for J0023+0923_synthetic_injected4 =  9.015925861272951 years.
enterprise v3.4.2,  Python v3.12.8

Signal Name                              Signal Class                   no. Parameters      
J0023+0923_marginalizing_linear_timing_model TimingModel                    0                   

params:
__________________________________________________________________________________________
J0023+0923_measurement_noise             MeasurementNoise               0                   

params:
J0023+0923_430_ASP_efac:Constant=1.101966732560332                                        
J0023+0923_430_ASP_log10_t2equad:Constant=-8.379328374813772                              
J0023+0923_430_PUPPI_efac:Constant=1.05440100253028                                       
J0023+0923_430_PUPPI_log10_t2equad:Constant=-6.795146207080774                            
J0023+0923_L-wide_ASP_efac:Constant=1.080482895771678                                     
J0023+0923_L-wide_ASP_log10_t2equad:Constant=-5.

In [3]:
5

5

In [4]:
import numpy as np
import json
import glob
import sys
import shutil

from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals.parameter import Uniform
from enterprise.signals import selections
from enterprise.signals import white_signals
from enterprise.signals.gp_signals import MarginalizingTimingModel
from enterprise_extensions import blocks
from troj_signal import troj_res_block
from enterprise.signals import signal_base

from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
import pint.models.model_builder as mb

pname = "J0023+0923"
mc_min = 0.016388
synthetic = False

print(f"Working on PSR {pname}.")

if synthetic:
    par = glob.glob(f"./red_noise_and_injected/{pname}.par")
    tim = glob.glob(f"./red_noise_and_injected/{pname}.tim")
    psr_real_name = "J0023+0923"
else:
    par = glob.glob(f"./data/par/{pname}_PINT_*.nb.par")
    tim = glob.glob(f"./data/tim/{pname}_PINT_*.nb.tim")
    psr_real_name = psr
    

if len(par) != 1 and len(tim) != 1:
    sys.exit("Exiting! More than one par/tim file present for this pulsar!")
psr = Pulsar(par[0], tim[0], timing_package="pint")
model = mb.get_model(par[0])
f_b = model.FB0.value # no factor of 86400 because ENTERPRISE uses units of seconds for TOAs
if f_b is None:
    P_b_days = model.PB.value
    P_b_seconds = P_b_days * 86400
    f_b = 1 / P_b_seconds

n_b_Const = parameter.Constant(f_b * np.pi*2)("n_b")
n_b = f_b * np.pi*2
# n_b = f_b * np.pi*2

kappa = (1.4 - mc_min) / (1.4 + mc_min)
nu_min = 1/10 * n_b/2 * np.sqrt(2 - np.sqrt(27*kappa**2 - 23))
nu_max = n_b * np.sqrt(2)/2*1.1

tmin = psr.toas.min()
tmax = psr.toas.max()
Tspan = tmax - tmin
print(f"Tspan for {pname} =  {Tspan/(365.25*86400)} years.")

# define selection by observing backend
selection = selections.Selection(selections.by_backend)

# white noise parameters
efac = parameter.Constant()
equad = parameter.Constant()
ecorr = parameter.Constant()



efeq = white_signals.MeasurementNoise(
    efac=efac, log10_t2equad=equad, selection=selection
)
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection)

tm = MarginalizingTimingModel(use_svd=True)
rn = blocks.red_noise_block(psd="powerlaw", components=30)
troj = troj_res_block(nu=Uniform(nu_min, nu_max), n_b = n_b_Const)

signal_model = tm + efeq + ec + rn + troj
pta = signal_base.PTA(signal_model(psr))

noisedict = json.load(open("./data/15year_noise_dict.json", "r"))

# set white noise parameters with dictionary
pta.set_default_params(noisedict)
print(pta.summary())
print(f"param names = {pta.param_names}")
print(f"nu_min = {nu_min}, nu_max = {nu_max}")

x0 = np.hstack([p.sample() for p in pta.params])
print(f"x0 = {x0}")
print(f"lnprior at x0 = {pta.get_lnprior(x0)}")
print(f"lnlikelihood at x0 = {pta.get_lnlikelihood(x0)}")

ndim = len(x0)

# set up the sampler:
# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2)



Working on PSR J0023+0923.


2025-02-14 16:14:15.079 | DEBUG    | pint.toa:get_TOAs:195 - Using EPHEM = DE440 from the given model
2025-02-14 16:14:15.080 | DEBUG    | pint.toa:get_TOAs:211 - Using CLOCK = BIPM2019 from the given model
2025-02-14 16:14:25.861 | DEBUG    | pint.toa:apply_clock_corrections:2224 - Applying clock corrections (include_bipm = True)
2025-02-14 16:14:27.795 | INFO     | pint.observatory:gps_correction:235 - Applying GPS to UTC clock correction (~few nanoseconds)
2025-02-14 16:14:27.798 | INFO     | pint.observatory:bipm_correction:250 - Applying TT(TAI) to TT(BIPM2019) clock correction (~27 us)
2025-02-14 16:14:27.800 | INFO     | pint.observatory.topo_obs:clock_corrections:340 - Applying observatory clock corrections for observatory='arecibo'.
2025-02-14 16:14:39.279 | DEBUG    | pint.toa:compute_TDBs:2270 - Computing TDB columns.
2025-02-14 16:14:39.280 | DEBUG    | pint.toa:compute_TDBs:2291 - Using EPHEM = DE440 for TDB calculation.
2025-02-14 16:14:43.017 | DEBUG    | pint.toa:get_TO

Tspan for J0023+0923 =  9.01592586048203 years.
enterprise v3.4.2,  Python v3.12.8

Signal Name                              Signal Class                   no. Parameters      
J0023+0923_marginalizing_linear_timing_model TimingModel                    0                   

params:
__________________________________________________________________________________________
J0023+0923_measurement_noise             MeasurementNoise               0                   

params:
J0023+0923_430_ASP_efac:Constant=1.101966732560332                                        
J0023+0923_430_ASP_log10_t2equad:Constant=-8.379328374813772                              
J0023+0923_430_PUPPI_efac:Constant=1.05440100253028                                       
J0023+0923_430_PUPPI_log10_t2equad:Constant=-6.795146207080774                            
J0023+0923_L-wide_ASP_efac:Constant=1.080482895771678                                     
J0023+0923_L-wide_ASP_log10_t2equad:Constant=-5.957285653521352      

In [5]:
import numpy as np
import json
import argparse
import glob
import sys
import shutil
from datetime import datetime

from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals.parameter import Uniform
from enterprise.signals import selections
from enterprise.signals import white_signals
from enterprise.signals.gp_signals import MarginalizingTimingModel
from enterprise_extensions import blocks
# from troj_signal_old import troj_res_block, troj_res_block_unif
from troj_signal_2nd import troj_res_block, troj_res_block_unif
from enterprise.signals import signal_base

from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
import pint.models.model_builder as mb

parser = argparse.ArgumentParser()
parser.add_argument("-pname", "--pulsar_name", type=str)
parser.add_argument("-mc_min", "--m_c_min_mass", type=float,)
parser.add_argument("-niter", "--num_of_iteration", default=int(1e6), type=int)
parser.add_argument("-B_lim", "--B_lim", default="", type=str)
parser.add_argument(
        "--synthetic",
        help="Whether NG 15yr or injected (synthetic) toas",
        action=argparse.BooleanOptionalAction,
        type=bool,
        default=False,
)

# args = parser.parse_args()
pname = "J0023+0923"
pname_files = pname
# niter = args.num_of_iteration
# mc_min = args.m_c_min_mass
# B_lim = args.B_lim
# if B_lim:
#     B_lim = args.B_lim.split(",")
#     B_lim = (float(B_lim[0]), float(B_lim[1]))
#     pname_files = pname[:-6]
    
# print(f"Working on PSR {pname}.")
# print(f"{B_lim=}")

if False:
    par = glob.glob(f"./red_noise_and_injected/{pname_files}.par")
    tim = glob.glob(f"./red_noise_and_injected/{pname_files}.tim")
    psr_real_name = "J0023+0923"
else:
    par = glob.glob(f"./data/par/{pname}_PINT_*.nb.par")
    tim = glob.glob(f"./data/tim/{pname}_PINT_*.nb.tim")
    psr_real_name = pname
    

if len(par) != 1 and len(tim) != 1:
    sys.exit("Exiting! More than one par/tim file present for this pulsar!")
psr = Pulsar(par[0], tim[0], timing_package="pint")
model = mb.get_model(par[0])
f_b = model.FB0.value # no factor of 86400 because ENTERPRISE uses units of seconds for TOAs
if f_b is None:
    P_b_days = model.PB.value
    P_b_seconds = P_b_days * 86400
    f_b = 1 / P_b_seconds

n_b_Const = parameter.Constant(f_b * np.pi*2)("n_b")
n_b = f_b * np.pi*2
# n_b = f_b * np.pi*2

m_p = 1.35
mu_max = 1/27 * 1.1
mu_min = mc_min / (mc_min + m_p) * 0.8
nu_min = n_b * np.sqrt(27/4 * mu_min)
# nu_max = n_b * np.sqrt(27/4 * mu_max)

# kappa = (1.4 - mc_min) / (1.4 + mc_min)
# nu_min = 1/10 * n_b/2 * np.sqrt(2 - np.sqrt(27*kappa**2 - 23))
nu_max = n_b * np.sqrt(2)/2*1.2 # this is larger than the other nu_max

tmin = psr.toas.min()
tmax = psr.toas.max()
Tspan = tmax - tmin
print(f"Tspan for {pname} =  {Tspan/(365.25*86400)} years.")

2025-06-10 20:44:28.824 | DEBUG    | pint.toa:get_TOAs:195 - Using EPHEM = DE440 from the given model
2025-06-10 20:44:28.825 | DEBUG    | pint.toa:get_TOAs:211 - Using CLOCK = BIPM2019 from the given model
2025-06-10 20:44:39.457 | DEBUG    | pint.toa:apply_clock_corrections:2224 - Applying clock corrections (include_bipm = True)
2025-06-10 20:44:41.303 | INFO     | pint.observatory:gps_correction:235 - Applying GPS to UTC clock correction (~few nanoseconds)
2025-06-10 20:44:41.304 | DEBUG    | pint.observatory:_load_gps_clock:109 - Loading global GPS clock file
2025-06-10 20:44:41.314 | INFO     | pint.observatory.global_clock_corrections:get_file:129 - File index.txt to be downloaded due to download policy if_expired: https://raw.githubusercontent.com/ipta/pulsar-clock-corrections/main/index.txt
2025-06-10 20:44:41.498 | DEBUG    | pint.observatory.clock_file:__init__:812 - Global clock file gps2utc.clk saving kwargs={'bogus_last_correction': False, 'valid_beyond_ends': False}
2025-

NameError: name 'mc_min' is not defined

tmin

In [7]:
tmin = psr.toas.min()
tmax = psr.toas.max()
Tspan = tmax - tmin
print(f"Tspan for {pname} =  {Tspan/(365.25*86400)} years.")

Tspan for J0023+0923 =  9.01592586048203 years.


In [8]:
tmin

4817521249.0561495